In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
import copy

In [2]:
# Parameter defination
T = 100
max_cache_size = 100
until = []
files = 1000
locations = 4
caches = 3
action_dim = files*locations*caches+files*caches
l = np.array([[1, 1, 0],
              [1, 1, 0],
              [0, 1, 1],
              [0, 1, 1]
              ])
utils = np.tile(np.array([1, 2, 100]), (1, files * locations))[0]

In [3]:
#other defination
z_dim = files * locations * caches
y_dim = files * caches
z = cp.Variable(z_dim)
y = cp.Variable(y_dim)
x = cp.hstack([z, y]) 
acc_grads = np.zeros(z_dim)

actions = []
grads = []
acc_sigma = 0
acc_sigma_action = 0
sigmas = []
etas = []
acc_grad_square = 0
R = np.sqrt(2 * caches * max_cache_size + 2)

In [4]:
# Constraints 
standard_constraints = []
# Contrains of y
standard_constraints.extend([y >= 0, y <= 1]) # Basic defination
y_consts_mat = []
for cache_idx in range(caches):
    const_coeff = np.zeros(files * caches)
    const_coeff[cache_idx::caches] = 1
    y_consts_mat.append(copy.deepcopy(const_coeff))
y_consts_mat = np.array([np.array(constraint) for constraint in y_consts_mat])
standard_constraints.extend([y_consts_mat @ y <= max_cache_size])
# Constrains of z
standard_constraints.extend([z >= 0, z <= 1]) # Defination of z
z_consts_mat = []
for ni in range(files * locations):
    start = ni * (caches)
    end = (ni + 1) * (caches)
    const_coeff = np.zeros(files * locations * caches)
    const_coeff[start: end] = 1
    z_consts_mat.append(copy.deepcopy(const_coeff))
z_consts_mat = np.array([np.array(constraint) for constraint in z_consts_mat])
standard_constraints.extend([z_consts_mat @ z <= 1]) 
# Constrains of min problem
fl = l.flatten()
extended_fl = np.concatenate([fl for i in range(files)]).ravel()
sl = []
for p in range(files):
    start = caches * p
    end = caches * (p + 1)
    for pp in range(locations):
        sl.append(y[start:end])
y_comp = cp.hstack(sl)
standard_constraints.extend([z <= cp.multiply(y_comp, extended_fl)])

In [5]:
# Optimization Problem
non_projected_sol_parameter = cp.Parameter(z_dim + y_dim)
objective = cp.Minimize(cp.sum_squares(x - non_projected_sol_parameter))
prob = cp.Problem(objective, standard_constraints)

In [6]:
# Functions
def cal_sigma(acc_grad_square,sigmas,etas,R,pred):
    if not sigmas:  # no grads yet
        etas.append(1)  # for t=0
        sigmas.append(1 / etas[-1])  # from McMahan \sigma_0  = 1 / \eta_0
    else:
        acc_grad_square += np.linalg.norm(grads[-1] - pred, 2) ** 2
        etas.append(np.sqrt(2) * R / np.sqrt(acc_grad_square))  # this will be the second eta, after 0
        sigmas.append(1 / etas[-1] - 1 / etas[-2])
    return acc_grad_square,sigmas,etas

def step(proba,actions,acc_sigma,acc_sigma_action,acc_grads, grad,etas,x,acc_grad_square):
    pred = get_prediction(proba,locations,files,caches,grad)
    

    if not actions:  # first action, minimize r_0, returning 0 because it assumed to be ||x||
        yy = np.random.rand(z_dim + y_dim)
        objective = cp.Minimize(cp.sum_squares(x - yy))
    else:
        # calculating x
        if acc_sigma > 0:
            padded_pred = np.pad(pred, (0, files * caches), 'constant')
            padded_acc_grad = np.pad(acc_grads, (0, files * caches), 'constant')
            # yy = (acc_sigma_action - (padded_acc_grad + padded_pred)) / acc_sigma 
            yy = (acc_sigma_action + (padded_acc_grad + padded_pred)) / acc_sigma  
            objective = cp.Minimize(cp.sum_squares(x - yy))

        else:   
            a = np.pad(acc_grads + pred, (0, files * caches), 'constant')
            objective = cp.Maximize(a @ x)
            

    # Projection
    #if acc_sigma > 0 or not actions:
    #    objective = cp.Minimize(cp.sum_squares(x - yy))
    #    non_projected_sol_parameter.value = yy
    #    result = prob.solve(warm_start=True)
    
    prob = cp.Problem(objective, standard_constraints)
    result = prob.solve(warm_start=True)
    x = cp.hstack([z, y])
    actions.append(x.value)
    

    # preparing params for next round
    grads.append(grad)
    acc_grads += grads[-1]
    acc_grad_square,new_sigmas,etas = cal_sigma(acc_grad_square,sigmas,etas,R,pred)
    acc_sigma += new_sigmas[-1]
    acc_sigma_action += new_sigmas[-1] * actions[-1]

    return x.value, z, y, grads,acc_grads, acc_sigma, acc_sigma_action, actions, etas, new_sigmas,x,acc_grad_square

In [7]:
def generate_c(locations,files, caches, utils):
    r = [np.zeros(locations) for i in range(files)]
    location = np.random.choice(range(locations))
    requested_id = np.random.randint(0,files)
    r[requested_id][location] = 1

    r = np.concatenate(r).ravel() 
    extended_r = np.repeat(r, caches)
    coeff = utils * extended_r
    return coeff

def get_prediction(proba,locations,files,caches,grad):  # grad is the true grad
    if np.random.random(1) <= proba:
        pred = grad
    else:
        r = [np.zeros(locations) for i in range(files)]
        location = np.random.choice(range(locations))
        requested_id = np.random.randint(low=0, high=files)

        r[requested_id][location] = 1
        r = np.concatenate(r).ravel()  
        extended_r = np.repeat(r, caches)
        coeff = utils * extended_r
        pred = coeff
    return pred
        


In [ ]:
# main function
proba = 0.8
util_OFTRL = []
for t in range(0,T):
    c_t = generate_c(locations,files, caches, utils)
    x,z,y,grads,acc_grads,acc_sigma,acc_sigma_action,actions,etas,sigmas,x,acc_grad_square= step(proba,actions,acc_sigma,acc_sigma_action,acc_grads,c_t,etas,x,acc_grad_square)
    if t == 0:
     z = z.value
    util_OFTRL.append(c_t @ z)
    print(util_OFTRL)

[-0.0002531939827428085]


/var/folders/75/f6xqg0dd3fn_kbtrm4x_1qd80000gn/T/ipykernel_19523/4096339829.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  etas.append(np.sqrt(2) * R / np.sqrt(acc_grad_square))  # this will be the second eta, after 0


[-0.0002531939827428085, 40.12848490264103]
[-0.0002531939827428085, 40.12848490264103, -0.0002485288687955774]
[-0.0002531939827428085, 40.12848490264103, -0.0002485288687955774, 0.24091708213757637]
[-0.0002531939827428085, 40.12848490264103, -0.0002485288687955774, 0.24091708213757637, 0.27071577443658057]
[-0.0002531939827428085, 40.12848490264103, -0.0002485288687955774, 0.24091708213757637, 0.27071577443658057, 38.51824934751311]
[-0.0002531939827428085, 40.12848490264103, -0.0002485288687955774, 0.24091708213757637, 0.27071577443658057, 38.51824934751311, 0.3182024781519434]
[-0.0002531939827428085, 40.12848490264103, -0.0002485288687955774, 0.24091708213757637, 0.27071577443658057, 38.51824934751311, 0.3182024781519434, 37.03323636381971]
[-0.0002531939827428085, 40.12848490264103, -0.0002485288687955774, 0.24091708213757637, 0.27071577443658057, 38.51824934751311, 0.3182024781519434, 37.03323636381971, 0.1999038070592619]
[-0.0002531939827428085, 40.12848490264103, -0.00024852

In [ ]:
util_OFTRL= np.cumsum(util_OFTRL) / np.arange(1, t + 1)
plt.figure(figsize=(6.5, 6.5))
plt.plot(np.arange(T - 1), util_OFTRL, color='Blue', label=r"OBC, $\rho=0.7$", linewidth=2.5, markevery=1000,marker='d', markersize=12, markeredgecolor='Grey')
plt.show()